In [16]:
import re
import os
import glob
import numpy as np
import pandas as pd

In [17]:
current_path = os.getcwd()
output_path = f"{current_path}/output"
csv_path = f"{output_path}/csv"
train_path = f"{output_path}/train"

In [18]:
train_csv_list = glob.glob(f"{csv_path}/*_train.csv")


In [19]:
test_csv_list=train_csv_list.pop(1)
print([test_csv_list])

['/Users/nakamura.yuta/Desktop/num_plate/car_number/output/csv/20190125_train.csv']


In [20]:
def read_csv_to_df(train_csv_list):
    columns = ["lu_x","lu_y","ru_x","ru_y","ld_x","ld_y","rd_x","rd_y"]
    return_dict = {}
    dir_list=[]
    df_concat = pd.Series([0,0,0,0,0,0,0,0],index = columns)
    for train_csv in train_csv_list:
        dir_path = train_csv.split('/')[-1].rstrip("_train.csv")
        read_df = pd.read_csv(train_csv,index_col=0)
        df_concat = pd.concat([df_concat, read_df.T],axis=1)
        return_dict.update({dir_path:df_concat.T[1:]})
        dir_list.append(dir_path)
        df_concat = pd.Series([0,0,0,0,0,0,0,0],index = columns)
    return return_dict,dir_list

In [21]:
read_csv_dict,dir_list = read_csv_to_df(train_csv_list)
print(read_csv_dict["001"])
print(read_csv_dict["001"].index.values)


                                                  lu_x        lu_y  \
36E1C953-455F-4189-834C-1818BFAA06FC-0       38.376709  152.560386   
36E1C953-455F-4189-834C-1818BFAA06FC-1       84.835165  143.457189   
4228DA2D-57A7-4505-BEE2-075AB1B7213B         39.004757  148.791894   
83C51BA3-3276-4A8C-95E3-07584027B025-0       75.630252  146.135831   
83C51BA3-3276-4A8C-95E3-07584027B025-2      140.283570  145.185185   
83C51BA3-3276-4A8C-95E3-07584027B025-3       50.082919  154.435484   
83C51BA3-3276-4A8C-95E3-07584027B025-5      149.373434  139.664804   
20763A20-D514-47B6-8E8D-D133ABA5EA83-1       47.136150  134.027778   
20763A20-D514-47B6-8E8D-D133ABA5EA83-2       78.488982  147.972973   
60819F66-AE67-4AE7-B1E5-7A230BD83135         50.793651  144.796784   
2EA974C7-0BE9-4A27-9CE2-03E722E17618-0       97.114428  125.154639   
2EA974C7-0BE9-4A27-9CE2-03E722E17618-2      165.770171  137.600000   
2EA974C7-0BE9-4A27-9CE2-03E722E17618-3      160.065200   46.400000   
5413DCE6-F523-4600-9

In [22]:
for name in dir_list:
    for val in read_csv_dict[name].index.values:
        print(f"{train_path}/{name}/{val}")
        img_path = f"{train_path}/{name}/{val}"

/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4414
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4399
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4416
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4412
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4406
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4407
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4413
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4405
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4411
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4388
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4389
/Users/nakamura.yuta/Desktop/num_plate/car_number/output/train/20190124/IMG_4410
/Users/nakamura.yuta/Desktop